# RAG Model for Book Question Answering

In [10]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv(override=True)

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

In [7]:
#send a request to the model
response = model.invoke("This is a test message to the Gemini 2.5 model.")
print(response)

content="Thank you for the test message! I've received it.\n\nAs Gemini 2.5, I'm ready to process your requests and engage in conversations. Please feel free to ask me anything, give me tasks, or provide me with information. I'm here to help!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--565b915e-1ecc-4ecf-8f86-a75ae621eaa2-0' usage_metadata={'input_tokens': 15, 'output_tokens': 59, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}}


In [8]:
#Check whether these invokations have a state
response = model.invoke("What did I ask you before?")
print(response)

content="I am a large language model, trained by Google. I don't have access to past conversations. Therefore, I cannot tell you what you asked me before." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--50bb29b5-3ae5-4ff0-9931-c3df5ad71414-0' usage_metadata={'input_tokens': 8, 'output_tokens': 33, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}}


In [ ]:
import requests
url = f"https://generativelanguage.googleapis.com/v1beta/models?key={os.environ['GOOGLE_API_KEY']}"

response = requests.get(url)
models = response.json()

for model_i in models.get('models', []):
    if 'embedContent' in model_i.get('supportedGenerationMethods', []):
        print(f"Embedding model: {model_i['name']}")

Embedding model: models/embedding-001
Embedding model: models/text-embedding-004
Embedding model: models/gemini-embedding-exp-03-07
Embedding model: models/gemini-embedding-exp
Embedding model: models/gemini-embedding-001


In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

#embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

#Free tier embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="text-embedding-004")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


# ---- Dataset ----
eng = [
    "The cat is sleeping.",
    "The dog is sleeping.",
    "The rocket is launching into space.",
    "I am cooking pasta in the kitchen.",
    "The economy is experiencing significant growth this quarter.",
    "The stock market reached an all-time high today.",
]

spa = [
    "El gato está durmiendo.",
    "El perro está durmiendo.",
    "El cohete está despegando hacia el espacio.",
    "Estoy cocinando pasta en la cocina.",
    "La economía está experimentando un crecimiento significativo este trimestre.",
    "El mercado de valores alcanzó un máximo histórico hoy.",
]

# ---- Embed ----
emb_eng = [embeddings.embed_query(s) for s in eng]
emb_spa = [embeddings.embed_query(s) for s in spa]

# ---- Similarity matrices ----
sim_eng = cosine_similarity(emb_eng)
sim_spa = cosine_similarity(emb_spa)

# ---- Compare the key pairs ----
pairs = [
    (0, 1, "Similar action (sleeping), different animal"),
    (2, 3, "Totally different topics"),
    (4, 5, "Similar topic (economy/stock market)"),
]

print("\n===== Similarity Comparison: English vs Spanish =====\n")

for i, j, label in pairs:
    print(f"--- {label} ---")
    print(f"EN ({eng[i]}) vs ({eng[j]}): {sim_eng[i][j]:.4f}")
    print(f"ES ({spa[i]}) vs ({spa[j]}): {sim_spa[i][j]:.4f}")
    diff = sim_eng[i][j] - sim_spa[i][j]
    print(f"Difference (EN - ES): {diff:.4f}\n")

#Looks like the model captures similarities somewhat consistently across both languages


===== Similarity Comparison: English vs Spanish =====

--- Similar action (sleeping), different animal ---
EN (The cat is sleeping.) vs (The dog is sleeping.): 0.8296
ES (El gato está durmiendo.) vs (El perro está durmiendo.): 0.8640
Difference (EN - ES): -0.0344

--- Totally different topics ---
EN (The rocket is launching into space.) vs (I am cooking pasta in the kitchen.): 0.3719
ES (El cohete está despegando hacia el espacio.) vs (Estoy cocinando pasta en la cocina.): 0.5021
Difference (EN - ES): -0.1303

--- Similar topic (economy/stock market) ---
EN (The economy is experiencing significant growth this quarter.) vs (The stock market reached an all-time high today.): 0.4961
ES (La economía está experimentando un crecimiento significativo este trimestre.) vs (El mercado de valores alcanzó un máximo histórico hoy.): 0.6235
Difference (EN - ES): -0.1274



In [11]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="../vectors/chroma_langchain_db",  # Where to save data locally
)

In [6]:
import bs4
from langchain_community.document_loaders import TextLoader

# Only keep post title, headers, and content from the full HTML.
loader = TextLoader(
    "../data/Guerra_y_paz_cleaned.txt",
    encoding="utf-8",
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 3153561


In [7]:
print(docs[0].page_content[:500])

—Eh bien, mon prince, Génova y Lucca ya no son más que posesiones de la familia Bonaparte. No, le prevengo que si usted no me dice que estamos en plena guerra, si vuelve a permitirse paliar todas las infamias, todas las atrocidades de ese Anticristo (le doy mi palabra de que así lo considero), a usted ya no lo conozco, no es usted mi amigo, no es mi devoto esclavo, como dice. Ea, bienvenido, bienvenido. Veo que lo he asustado. Siéntese y charlemos.
Con tales palabras, Anna Pávlovna Scherer, dama


In [13]:
type(docs[0])

langchain_core.documents.base.Document

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_0 = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=0,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

text_splitter_200 = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits_0 = text_splitter_0.split_documents(docs)
all_splits_200 = text_splitter_200.split_documents(docs)
print(f"Split blog post into {len(all_splits_0)} sub-documents.")
print(f"Split blog post into {len(all_splits_200)} sub-documents.")

Split blog post into 4140 sub-documents.
Split blog post into 4387 sub-documents.


In [9]:
split_0_200 = all_splits_200[0].page_content
split_1_200 = all_splits_200[1].page_content
split_2_200 = all_splits_200[2].page_content
split_3_200 = all_splits_200[3].page_content
split_4_200 = all_splits_200[4].page_content

split_0_0 = all_splits_0[0].page_content
split_1_0 = all_splits_0[1].page_content
split_2_0 = all_splits_0[2].page_content
split_3_0 = all_splits_0[3].page_content
split_4_0 = all_splits_0[4].page_content

In [10]:
assert split_0_0 == split_0_200
assert split_1_0 == split_1_200
assert split_2_0 == split_2_200
assert split_3_0 == split_3_200
assert split_4_0 == split_4_200

AssertionError: 

In [11]:
print("Len of split 0: 200 overlap", len(split_0_200))
print("Len of split 0: 000 overlap", len(split_0_0))

print("Len of split 1: 200 overlap", len(split_1_200))
print("Len of split 1: 000 overlap", len(split_1_0))

print("Len of split 2: 200 overlap", len(split_2_200))
print("Len of split 2: 000 overlap", len(split_2_0))

print("Len of split 3: 200 overlap", len(split_3_200))
print("Len of split 3: 000 overlap", len(split_3_0))

print("Len of split 4: 200 overlap", len(split_4_200))
print("Len of split 4: 000 overlap", len(split_4_0))

# There seems to be a faulty behavior with the implementation of this splitter.
# The overlaps seem to only be launched when the original split is smaller than chunk_size - chunk_overlap or something.
# Need to investigate further. https://stackoverflow.com/questions/76681318/why-is-recursivecharactertextsplitter-not-giving-any-chunk-overlap

Len of split 0: 200 overlap 974
Len of split 0: 000 overlap 974
Len of split 1: 200 overlap 874
Len of split 1: 000 overlap 874
Len of split 2: 200 overlap 944
Len of split 2: 000 overlap 944
Len of split 3: 200 overlap 532
Len of split 3: 000 overlap 967
Len of split 4: 200 overlap 541
Len of split 4: 000 overlap 999


In [12]:
document_ids = vector_store.add_documents(documents=all_splits_200)

print(document_ids[:3])

GoogleGenerativeAIError: Error embedding content: 404 models/text-multilingual-embedding-002 is not found for API version v1beta, or is not supported for embedContent. Call ListModels to see the list of available models and their supported methods.

In [12]:
# Now you can query it directly
results = vector_store.similarity_search("¿Quien es Napoleón?", k=5)

In [13]:
results

[Document(id='0aebe282-f92e-4235-a907-bc77797fdf74', metadata={'source': '../data/Guerra_y_paz_cleaned.txt', 'start_index': 2931087}, page_content='Y a medida que esas fuerzas se multiplican aumenta más el prestigio del hombre que lo dirige y más justificadas se consideran sus acciones. Durante el período preparatorio de diez años que precede al gran movimiento, ese hombre se emparenta con todas las cabezas coronadas de Europa. Los desbancados dueños del mundo no pueden oponer ideal alguno razonable al insensato ideal de gloria y grandeza de Napoleón. Uno tras otro, se apresuran a demostrarle lo poco que valen. El rey de Prusia envía a su esposa para lograr el favor del gran hombre. El emperador de Austria considera un honor que ese hombre acepte en su lecho a la hija de los Césares. El Papa, custodio de los santuarios de los pueblos, pone en juego la religión para exaltarlo. No es Napoleón solo quien se prepara para asumir su papel; los que lo rodean, hasta más que él, lo preparan par

In [14]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def get_similar_documents(query: str):
    """Retrieve passages from the book to help answer a question."""
    retrieved_docs = vector_store.similarity_search(query, k=3)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [15]:
from langchain.agents import create_agent


tools = [get_similar_documents]

system_prompt = (
    "You are an AI assistant for answering questions about a book or document. "
    "You have access to a tool that retrieves context from a database of books you have available to read."
    "Use the tool to help answer user queries."
    "If you use the tool, be sure to cite the sources of the information you provide."
    "At the end of the explanation make a little summarty starting 'In summary,...'"
)
agent = create_agent(model=model, tools=tools, system_prompt=system_prompt)

In [65]:
model.input_schema

langchain_google_genai.chat_models.ChatGoogleGenerativeAIInput

In [72]:
#agent.invoke({"question": "¿Quién es Napoleón en el libro Guerra y Paz?"})

query = (
    "¿Quién es Napoleón en el libro Guerra y Paz?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

¿Quién es Napoleón en el libro Guerra y Paz?
================================== Ai Message ==================================
Tool Calls:
  get_similar_documents (b433bbaf-8cdd-4070-94a1-604e128206d4)
 Call ID: b433bbaf-8cdd-4070-94a1-604e128206d4
  Args:
    query: Napoleón en Guerra y Paz
================================= Tool Message =================================
Name: get_similar_documents

Source: {'source': '../data/Guerra_y_paz_cleaned.txt', 'start_index': 694379}
Content: Aquél era para Napoleón un día solemne: el aniversario de su coronación. Antes del alba había dormido unas horas, y ahora, tranquilo, jovial, descansado, en esa feliz disposición de ánimo en la que todo parece posible y todo se consigue, había montado a caballo para dirigirse al campo de batalla. Permanecía inmóvil, mirando hacia las colinas que se iban liberando de la niebla; su rostro frío reflejaba aquel matiz peculiar de 

In [16]:
response = agent.invoke({"messages": [{"role": "user", "content": "¿Quién es Napoleón en el libro Guerra y Paz?"}]})

In [22]:
response["messages"][-1].content

'En "Guerra y Paz", Napoleón es presentado como una figura central, aunque a menudo distante, en los eventos históricos que se desarrollan en la novela. Se le describe en momentos clave, como en el campo de batalla, donde su presencia irradia confianza y determinación. Por ejemplo, en el aniversario de su coronación, se le ve tranquilo y seguro de sí mismo, observando el amanecer antes de dirigirse a la batalla, con la convicción de que la victoria es posible.\n\nTambién se muestra su faceta de líder, dando órdenes y motivando a sus tropas. En una de sus intervenciones, Napoleón insta a sus soldados a no romper filas y a vencer a los "mercenarios de Inglaterra", con la promesa de un regreso a los cuarteles de invierno y una paz digna.\n\nSin embargo, la novela también expone un lado más personal y, a veces, vanidoso de Napoleón. En una interacción, se le describe como alguien que busca engrandecer su propia persona y ofender a Alejandro, mostrando una elocuencia y una irritación conten

In [62]:
model

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B4F852AB50>, default_metadata=(), model_kwargs={})